<a href="https://colab.research.google.com/github/ehcaw/malimage/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project Code

## Dataset

In [ ]:
!pip install kagglehub

In [ ]:
import kagglehub
# Download latest version
# add '/Data' to the end to properly access the underlying images
DATA_DIR = kagglehub.dataset_download("satyaprakash138/balanced-malware-image-dataset") + "/Data"

print("Path to dataset files:", DATA_DIR)

Using Colab cache for faster access to the 'balanced-malware-image-dataset' dataset.
Path to dataset files: /kaggle/input/balanced-malware-image-dataset/Data


### ML Library Dependencies

In [ ]:
!pip install -q skorch

In [ ]:
import os
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets, transforms, models

from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
IMAGE_SIZE = 128


transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

train_dir = os.path.join(DATA_DIR, "train")
test_dir = os.path.join(DATA_DIR, "test")
val_dir = os.path.join(DATA_DIR, "val")

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)

print("Train classes:", train_dataset.classes)
print("Val   classes:", val_dataset.classes)
print("Test  classes:", test_dataset.classes)
print("Num train:", len(train_dataset), "Num val:", len(val_dataset), "Num test:", len(test_dataset))

num_classes = len(train_dataset.classes)



Train classes: ['MaliciousImages', 'NormalImages']
Val   classes: ['MaliciousImages', 'NormalImages']
Test  classes: ['MaliciousImages', 'NormalImages']
Num train: 17066 Num val: 2438 Num test: 4876


In [ ]:
def dataset_to_numpy(ds):
  X_list, y_list = [], []
  for img, label in ds:
    X_list.append(img.numpy())
    y_list.append(label)
  X = np.stack(X_list).astype("float32")
  y = np.array(y_list).astype("int64")

  return X, y


In [ ]:
X_train, y_train = dataset_to_numpy(train_dataset)
X_val, y_val = dataset_to_numpy(val_dataset)
X_test, y_test = dataset_to_numpy(test_dataset)

print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_val:",   X_val.shape,   "y_val:",   y_val.shape)
print("X_test:",  X_test.shape,  "y_test:",  y_test.shape)